In [1]:
from bs4 import BeautifulSoup 

import bs4 as bs
import pandas as pd
import requests 

In [2]:
resp = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
nh_toronto = pd.read_html(str(table))[0] # must have [0], not sure why 
   

nh_toronto.head()
     



,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [3]:
# clean the data
# Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
df = nh_toronto[nh_toronto.Borough !='Not assigned']
df.head()
df.iloc[5:9,]

,Postcode,Borough,Neighbourhood
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Not assigned
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge


In [4]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough. 
#df['Neighbourhood'].apply(lambda x: df.Borough if x=='Not assigned')

df.loc[(df.Neighbourhood == 'Not assigned'),'Neighbourhood']=df.Borough
df.iloc[5:9,]

/Users/wangshuang/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/Users/wangshuang/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,Postcode,Borough,Neighbourhood
7,M6A,North York,Lawrence Manor
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,Rouge


In [5]:
# More than one neighborhood in one postal code area will be combined into one row with the neighborhoods separated with a comma 
# reference: https://stackoverflow.com/questions/36274014/combine-rows-of-multiindex-dataframe-into-comma-separated-lists
df.set_index(['Postcode','Borough'],inplace=True)
df1 = df.groupby(level=['Postcode','Borough'], sort=False).agg( ','.join)
df1.head()

,,Neighbourhood
Postcode,Borough,
M3A,North York,Parkwoods
M4A,North York,Victoria Village
M5A,Downtown Toronto,"Harbourfront,Regent Park"
M6A,North York,"Lawrence Heights,Lawrence Manor"
M7A,Queen's Park,Queen's Park


In [6]:
df1.reset_index(inplace=True)
df1.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront,Regent Park"
3,M6A,North York,"Lawrence Heights,Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


explanation and assumptions:
1. scrape the table from the website using Beautifulsoup
2. Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned
3. Replace the "Not assigned" neighborhood with the name of the Borough.
to do this, first we need to set index with Borough and Neighborhood, and remove index afterwards. 



In [7]:
df1.shape

(103, 3)

In [15]:
# obtain the geographical coordinates of each postal code
postcode = pd.read_csv('Geospatial_Coordinates.csv')
postcode.head()
df2 = pd.merge(df1, postcode, left_on='Postcode', right_on='Postal Code').drop(['Postal Code'], axis=1)
df2.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront,Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights,Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
